In [1]:
import pandas as pd

In [2]:
df_final1=pd.read_csv('/content/drive/Shareddrives/CSCI567/CSCI567/writing_quality/final_df.csv')
merged_data= pd.read_csv('/content/drive/Shareddrives/CSCI567/CSCI567/writing_quality/train_logs.csv')

numerical_cols = ['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']
df_numerical_agg = merged_data.groupby('id')[numerical_cols].mean().reset_index()

# Handling categorical columns by getting the count of unique values
categorical_cols = ['activity', 'down_event', 'up_event', 'text_change']
df_categorical_agg = merged_data.groupby('id')[categorical_cols].agg(lambda x: x.value_counts().max()).reset_index()

# Merging the aggregated numerical and categorical DataFrames
df_final = pd.merge(df_numerical_agg, df_categorical_agg, on='id', how="outer")

# Merging the aggregated DataFrame with the scores DataFrame to get the labels
df_final = pd.merge(df_final, df_final1, on='id', how="outer")

df_final.head()

,id,down_time,up_time,action_time,cursor_position,word_count,activity,down_event,up_event,text_change,...,Total_Pause_Time_within_word,Mean_Pause_Time_within_word,SD_Pause_Time_within_word,Total_Pause_Time_Before_Sentences,Mean_Pause_Time_Before_Sentences,SD_Pause_Time_Before_Sentences,Total_Pause_Time_in_sentences,Mean_Pause_Time_in_sentences,SD_Pause_Time_in_sentences,score
0,001519c8,848180.771998,848297.018772,116.246774,711.163473,128.116152,2010,1619,1619,1940,...,685007,1918.787115,7273.768138,3643,173.476190,416.665168,1181819,56277.095238,43455.913672,3.5
1,0022f953,518855.347596,518967.568867,112.221271,776.205786,182.714751,1938,1490,1490,1698,...,1087710,2781.867008,16954.357992,69412,4627.466667,8278.455203,581673,38778.200000,32758.883810,3.5
2,0042269b,828491.775145,828593.612911,101.837766,731.611702,194.772727,3515,2904,2899,3257,...,843679,1528.403986,9793.226830,87089,4147.095238,15826.690943,1259632,59982.476190,58697.726407,6.0
3,0059420b,785483.026350,785604.874679,121.848329,542.537275,103.618895,1304,1038,1038,1146,...,745392,3067.456790,13400.325753,1466851,112834.692308,365509.369147,1032626,79432.769231,48355.056728,2.0
4,0075873a,713354.197550,713478.141446,123.943896,600.050968,125.082971,1942,1541,1541,1964,...,1039641,3208.768519,11181.242502,142133,6179.695652,13762.720184,1073221,46661.782609,73950.739406,4.0


In [3]:
# df_final1=pd.read_csv('/content/drive/Shareddrives/CSCI567/CSCI567/writing_quality/final_df.csv')
# df_final = pd.merge(df_final, df_final1[['id','score']], on='id', how="outer")
# df_final.head()

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import torch

# Assuming df_final is already defined and has the 'score' column

# Define features (X) and labels (y)
X = df_final.drop(columns=['id', 'score'])
y = df_final['score']
X_ids = df_final['id']

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Encode the labels to integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Splitting the encoded labels and scaled features into training and test sets (80% training, 20% test)
X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42
)


In [5]:
# from sklearn.preprocessing import MinMaxScaler
# sc = MinMaxScaler()

# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

# # Applying PCA function on training
# # and testing set of X component
# from sklearn.decomposition import PCA

# pca = PCA(n_components = 9)

# X_train = pca.fit_transform(X_train)
# X_test = pca.transform(X_test)

# explained_variance = pca.explained_variance_ratio_
# print(explained_variance)

# sum=0
# for i in explained_variance:
#     sum+=i

# print(sum)

In [6]:
# Convert the split labels and features to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_encoded, dtype=torch.long)
y_test_tensor = torch.tensor(y_test_encoded, dtype=torch.long)

l = y_train_tensor.unique().size(dim=0)

# Display the shapes to confirm the split
X_train.shape, X_test.shape, y_train_encoded.shape, y_test_encoded.shape

((1976, 58), (495, 58), (1976,), (495,))

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

# Defining the model
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = self.linear(x)
        # outputs = nn.functional.softmax(outputs, dim=1)
        return outputs

# Number of features and output classes
input_dim = X_train.shape[1]
output_dim = l

# Instantiate the model
model = LogisticRegressionModel(input_dim, output_dim)

# # Check for cuda
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# # Move the model to the device
# model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    # -(torch.nn.functional.log_softmax(y_pred, dim=1) * y_true).sum(dim=1).mean()
    # torch.nn.functional.cross_entropy(input=y_pred, target=y_true)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Display the model parameters
model.state_dict()

Epoch [100/1000], Loss: 2.1000
Epoch [200/1000], Loss: 1.9683
Epoch [300/1000], Loss: 1.8936
Epoch [400/1000], Loss: 1.8414
Epoch [500/1000], Loss: 1.8021
Epoch [600/1000], Loss: 1.7713
Epoch [700/1000], Loss: 1.7462
Epoch [800/1000], Loss: 1.7251
Epoch [900/1000], Loss: 1.7070
Epoch [1000/1000], Loss: 1.6912


OrderedDict([('linear.weight',
              tensor([[ 5.4722e-02,  8.6035e-02, -2.8766e-02,  8.3697e-02, -2.0347e-01,
                        1.3681e-01,  4.7482e-02,  2.1099e-01,  2.9471e-02,  5.7185e-02,
                        8.4939e-02, -1.0211e-01,  1.3674e-02,  5.8244e-02, -4.7226e-02,
                        4.6297e-02,  1.1282e-01,  1.5653e-02, -6.4555e-02, -1.3724e-02,
                        2.7584e-01, -1.9801e-01,  1.2319e-01, -7.0082e-02, -5.7749e-02,
                       -5.1069e-02, -6.5864e-03,  3.6299e-02, -1.1001e-01,  9.4047e-02,
                        2.5684e-02,  8.8881e-02, -2.0266e-01,  7.5984e-02,  5.7457e-02,
                        6.4479e-02,  9.7989e-02, -4.0706e-01,  4.3630e-01,  2.1344e-01,
                        5.9753e-02, -2.6522e-01,  4.3193e-01,  8.0831e-02,  1.1255e-01,
                       -7.7867e-02,  2.1428e-02, -6.8400e-02,  1.8010e-03, -1.5884e-01,
                        7.9736e-02,  1.1302e-01, -1.0824e-01,  1.2571e-01,  2.6139e-02,
 

In [8]:
# Evaluation phase
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Inference without gradient calculation
    correct = 0
    total = 0
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs.data, 1)
    total += y_test_tensor.size(0)
    correct += (predicted == y_test_tensor).sum().item()

print(correct)
accuracy = 100 * correct / total
print(f'Accuracy of the model on the test set: {accuracy:.2f}%')
print(f'Predicted classes: {predicted}')



157
Accuracy of the model on the test set: 31.72%
Predicted classes: tensor([ 5,  7,  5,  7,  4,  7,  7,  6,  8,  8,  5,  6,  5,  7,  8,  6,  8,  8,
         7,  5,  8,  6,  5, 10,  6,  6,  5,  7,  5,  6,  7,  7,  9,  8, 10,  5,
         5,  5,  6,  8,  5,  8,  8,  6,  7,  6,  4,  4,  8,  5,  5, 10,  8,  5,
         3,  6,  5,  6,  8,  6,  8,  7,  5,  6,  6,  8,  7,  5,  8,  7,  8,  7,
         5,  5,  8,  4,  7,  8,  5,  4,  7, 10,  5,  8,  4,  6,  9,  6,  8,  9,
         6,  7,  6,  8,  7,  6,  7, 10,  6,  8,  8,  5,  6,  8,  5,  7,  7,  5,
         5,  8,  6,  6,  6,  5,  5,  8,  5,  8,  8,  6,  6,  6,  7,  6, 10,  7,
         8,  6,  8,  8,  4,  4,  7,  7,  7,  6,  7,  5,  7,  6,  7,  5,  5,  8,
         7,  8,  7,  7, 10,  6,  4,  8,  7,  5,  7,  6,  6,  8,  6,  5,  6,  6,
         7,  8,  4, 10,  6,  4,  6,  5,  5,  4, 10,  8,  7,  8,  7,  6,  9, 10,
         8,  7,  8, 10,  6,  4,  6,  8,  7,  8,  8,  4,  8, 10, 10,  8,  8,  7,
         8,  4,  8,  7,  9,  5,  7, 10,  8,  4,  6,

In [ ]:
import numpy as np

# Convert predicted classes back to original scores using the inverse transform method of label_encoder
predicted_scores = label_encoder.inverse_transform(predicted.numpy())
y_scores = label_encoder.inverse_transform(y_test_encoded)
np.sqrt(np.mean((predicted_scores-y_scores)**2))

0.6834996102206037

In [ ]:
###################ORDINAL####################
import torch
import torch.nn as nn
import torch.optim as optim

class OrdinalLogisticRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(OrdinalLogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim - 1)

    def forward(self, x):
        logits = self.linear(x)
        return logits

def ordinal_logistic_loss(outputs, target, num_classes):
    # Creating a binary matrix for cumulative probabilities
    target = torch.nn.functional.one_hot(target, num_classes).cumsum(dim=1)[:, :-1].float()
    return torch.nn.functional.binary_cross_entropy_with_logits(outputs, target)

# Instantiate the ordinal logistic regression model
model = OrdinalLogisticRegressionModel(input_dim, output_dim)

# Use the custom ordinal logistic loss function
criterion = ordinal_logistic_loss

optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor, 12)
    # -(torch.nn.functional.log_softmax(y_pred, dim=1) * y_true).sum(dim=1).mean()
    # torch.nn.functional.cross_entropy(input=y_pred, target=y_true)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


# Evaluation phase
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    outputs = model(X_test_tensor)
    predicted = torch.sigmoid(outputs).cumsum(dim=1).argmax(dim=1)
    total += y_test_tensor.size(0)
    correct += (predicted == y_test_tensor).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the test set: {accuracy:.2f}%')

# Converting predicted classes back to original scores
predicted_scores = label_encoder.inverse_transform(predicted.numpy())
y_scores = label_encoder.inverse_transform(y_test_encoded)
np.sqrt(np.mean((predicted_scores - y_scores)**2))


Epoch [100/1000], Loss: 0.6018
Epoch [200/1000], Loss: 0.5589
Epoch [300/1000], Loss: 0.5263
Epoch [400/1000], Loss: 0.4985
Epoch [500/1000], Loss: 0.4740
Epoch [600/1000], Loss: 0.4520
Epoch [700/1000], Loss: 0.4321
Epoch [800/1000], Loss: 0.4141
Epoch [900/1000], Loss: 0.3978
Epoch [1000/1000], Loss: 0.3829
Accuracy of the model on the test set: 5.25%


2.03529463282942